In [3]:
import numpy as np 
import pandas as pd 
import xgboost as xgb 
from sklearn.cross_validation import train_test_split 

#记录程序运行时间 
import time 
start_time = time.time() 

#读入数据 
train = pd.read_csv("/home/zhangbo39/Downloads/mnist_train.csv") 
tests = pd.read_csv("/home/zhangbo39/Downloads/mnist_test.csv") 

train.shape, tests.shape


((42000, 785), (28000, 784))

In [23]:
train_xy, val = train_test_split(train, test_size=0.3)
train_y = train_xy.label
train_x = train_xy.drop(['label'], axis=1)
val_y = val.label
val_x = val.drop(['label'], axis=1)

xgb_train = xgb.DMatrix(train_x, label=train_y)
xgb_val = xgb.DMatrix(val_x, label=val_y)
xgb_test = xgb.DMatrix(tests)

/home/zhangbo39/anaconda3/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/zhangbo39/anaconda3/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [30]:
params={ 'booster':'gbtree', 
        'objective': 'multi:softmax', #多分类的问题 
        'num_class':10, # 类别数，与 multisoftmax 并用 
        #'eval_metric': 'auc',
        'seed':1,  
        'gamma':0.1, # 用于控制是否后剪枝的参数,越大越保守，一般0.1、0.2这样子。 
        'max_depth':12, # 构建树的深度，越大越容易过拟合 
        'lambda':2, # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。 
        'subsample':1, #0.7, # 随机采样训练样本 
        'colsample_bytree':1, #0.7, # 生成树时进行的列采样 
        'min_child_weight':1, #3, 
        # 这个参数默认是 1，是每个叶子里面 h 的和至少是多少，对正负样本不均衡时的 0-1 分类而言 
        #，假设 h 在 0.01 附近，min_child_weight 为 1 意味着叶子节点中最少需要包含 100 个样本。 
        #这个参数非常影响结果，控制叶子节点中二阶导的和的最小值，该参数值越小，越容易 overfitting。 
        'silent':0, #设置成1则没有运行信息输出，最好是设置为0. 
        'eta': 0.3, #0.007, # 如同学习率 
        
        'nthread':8#7 # cpu 线程数 #
        
       } 


plst = list(params.items()) 
num_rounds = 1000 # 迭代次数 
watchlist = [(xgb_train, 'train'),(xgb_val, 'val')] #训练模型并保存 

In [31]:
#early_stopping_rounds 当设置的迭代次数较大时，early_stopping_rounds 可在一定的迭代次数内准确率没有提升就停止训练 
model = xgb.train(plst, xgb_train, num_rounds, watchlist,early_stopping_rounds=100) 
#model = xgb.train(params, xgb_train, num_rounds, watchlist,early_stopping_rounds=100) 

[0]	train-merror:0.045816	val-merror:0.10881
Multiple eval metrics have been passed: 'val-merror' will be used for early stopping.

Will train until val-merror hasn't improved in 100 rounds.
[1]	train-merror:0.025646	val-merror:0.077857
[2]	train-merror:0.019354	val-merror:0.066905
[3]	train-merror:0.01398	val-merror:0.062063
[4]	train-merror:0.010272	val-merror:0.057857
[5]	train-merror:0.008095	val-merror:0.055635
[6]	train-merror:0.006599	val-merror:0.053413
[7]	train-merror:0.00483	val-merror:0.050873
[8]	train-merror:0.003299	val-merror:0.048889
[9]	train-merror:0.002619	val-merror:0.047619
[10]	train-merror:0.001803	val-merror:0.046905
[11]	train-merror:0.001497	val-merror:0.046587
[12]	train-merror:0.001122	val-merror:0.044921
[13]	train-merror:0.000918	val-merror:0.044683
[14]	train-merror:0.000612	val-merror:0.043889
[15]	train-merror:0.000476	val-merror:0.043571
[16]	train-merror:0.000374	val-merror:0.043095
[17]	train-merror:0.000238	val-merror:0.042381
[18]	train-merror:0.0

XGBoostError: b'[17:38:14] /workspace/dmlc-core/src/io/local_filesys.cc:209: Check failed: allow_null  LocalFileSystem::Open "./model/xgb.model": No such file or directory\n\nStack trace returned 10 entries:\n[bt] (0) /home/zhangbo39/anaconda3/xgboost/libxgboost.so(dmlc::StackTrace(unsigned long)+0x47) [0x7f4359fedfc7]\n[bt] (1) /home/zhangbo39/anaconda3/xgboost/libxgboost.so(dmlc::io::LocalFileSystem::Open(dmlc::io::URI const&, char const*, bool)+0x411) [0x7f435a191dd1]\n[bt] (2) /home/zhangbo39/anaconda3/xgboost/libxgboost.so(dmlc::Stream::Create(char const*, char const*, bool)+0x3a) [0x7f435a13e6fa]\n[bt] (3) /home/zhangbo39/anaconda3/xgboost/libxgboost.so(XGBoosterSaveModel+0x2b) [0x7f4359fe66eb]\n[bt] (4) /home/zhangbo39/anaconda3/lib/python3.6/lib-dynload/../../libffi.so.6(ffi_call_unix64+0x4c) [0x7f439b41aec0]\n[bt] (5) /home/zhangbo39/anaconda3/lib/python3.6/lib-dynload/../../libffi.so.6(ffi_call+0x22d) [0x7f439b41a87d]\n[bt] (6) /home/zhangbo39/anaconda3/lib/python3.6/lib-dynl

In [32]:
model.save_model('./xgb.model') # 用于存储训练出的模型 
print("best best_ntree_limit",model.best_ntree_limit)

best best_ntree_limit 58


In [33]:
preds = model.predict(xgb_test,ntree_limit=model.best_ntree_limit) 
np.savetxt('xgb_submission.csv',np.c_[range(1,len(tests)+1),preds],delimiter=',',header='ImageId,Label',comments='',fmt='%d') 

In [34]:
params={ 'booster':'gbtree', 
        'objective': 'multi:softmax', #多分类的问题 
        'num_class':10, # 类别数，与 multisoftmax 并用 
        #'eval_metric': 'auc',
        'seed':1,  
        'gamma':0.1, # 用于控制是否后剪枝的参数,越大越保守，一般0.1、0.2这样子。 
        'max_depth':8, # 构建树的深度，越大越容易过拟合 
        'lambda':2, # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。 
        'subsample':1, #0.7, # 随机采样训练样本 
        'colsample_bytree':1, #0.7, # 生成树时进行的列采样 
        'min_child_weight':1, #3, 
        # 这个参数默认是 1，是每个叶子里面 h 的和至少是多少，对正负样本不均衡时的 0-1 分类而言 
        #，假设 h 在 0.01 附近，min_child_weight 为 1 意味着叶子节点中最少需要包含 100 个样本。 
        #这个参数非常影响结果，控制叶子节点中二阶导的和的最小值，该参数值越小，越容易 overfitting。 
        'silent':0, #设置成1则没有运行信息输出，最好是设置为0. 
        'eta': 0.1, #0.007, # 如同学习率 
        
        'nthread':7 # cpu 线程数 #
        
       } 

num_rounds = 5000 # 迭代次数 
watchlist = [(xgb_train, 'train'),(xgb_val, 'val')] #训练模型并保存 
model = xgb.train(params, xgb_train, num_rounds, watchlist,early_stopping_rounds=100) 

[0]	train-merror:0.078401	val-merror:0.119365
Multiple eval metrics have been passed: 'val-merror' will be used for early stopping.

Will train until val-merror hasn't improved in 100 rounds.
[1]	train-merror:0.063878	val-merror:0.098333
[2]	train-merror:0.050034	val-merror:0.086429
[3]	train-merror:0.044422	val-merror:0.078413
[4]	train-merror:0.040442	val-merror:0.072937
[5]	train-merror:0.03932	val-merror:0.070317
[6]	train-merror:0.036361	val-merror:0.068968
[7]	train-merror:0.034116	val-merror:0.067222
[8]	train-merror:0.032143	val-merror:0.064841
[9]	train-merror:0.030578	val-merror:0.064365
[10]	train-merror:0.028503	val-merror:0.064048
[11]	train-merror:0.026633	val-merror:0.062222
[12]	train-merror:0.025408	val-merror:0.061349
[13]	train-merror:0.024082	val-merror:0.060794
[14]	train-merror:0.022789	val-merror:0.059683
[15]	train-merror:0.021531	val-merror:0.058413
[16]	train-merror:0.020238	val-merror:0.058254
[17]	train-merror:0.018946	val-merror:0.057698
[18]	train-merror:0